In [15]:
! sudo pip install snowflake-ml-python==1.0.5 -U
! pip install snowflake-snowpark-python==1.6.1 -U

     |████████████████████████████████| 327kB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 808kB 93kB/s s eta 0:00:01
     |████████████████████████████████| 71kB 47kB/s s eta 0:00:01
     |████████████████████████████████| 737kB 37.5MB/s eta 0:00:01
     |████████████████████████████████| 24.6MB 61.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 87.9MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 79.9MB/s eta 0:00:01
     |████████████████████████████████| 143kB 112.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 3.6MB/s  eta 0:00:01
     |████████████████████████████████| 204kB 114.4MB/s eta 0:00:01
     |████████████████████████████████| 194kB 105.9MB/s eta 0:00:01
     |████████████████████████████████| 450kB 107.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 118.9MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 88.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 1.6MB/s  eta 0

In [2]:
! pip install --upgrade xgboost==1.7.3

     |████████████████████████████████| 193.6MB 17kB/s /s eta 0:00:01
     |████████████████████████████████| 17.3MB 82.1MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 102.1MB/s eta 0:00:01


In [4]:
! pip list

Package                    Version     
-------------------------- ------------
absl-py                    1.4.0       
aiohttp                    3.8.5       
aiosignal                  1.3.1       
alembic                    1.11.1      
anyio                      3.7.0       
argon2-cffi                21.3.0      
argon2-cffi-bindings       21.2.0      
asn1crypto                 1.5.1       
asttokens                  2.2.1       
async-generator            1.10        
async-timeout              4.0.3       
attrs                      23.1.0      
Babel                      2.12.1      
backcall                   0.2.0       
beautifulsoup4             4.12.2      
bleach                     4.1.0       
boltons                    23.0.0      
bqplot                     0.12.39     
brotlipy                   0.7.0       
certifi                    2023.7.22   
certipy                    0.1.3       
cffi                       1.15.1      
charset-normalizer         3.2.0       


In [1]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.linear_model import SGDClassifier
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [3]:
session = get_session()

# Model Building
### Load the final data from snowflake table

In [4]:
table_name = 'predictive_maintenance_final'

In [5]:
sf_df = session.table(table_name).drop('ROW')

In [6]:
sf_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

In [8]:
a = sf_df.to_pandas()

In [11]:
list(a.columns)

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

In [10]:
a.head()

,FAILURE,METRIC5,METRIC6,DAY,DAY_WEEK,SECTOR,OP_PERIOD,DEV_RECONNECTED,MNW1,DIF_M6,DIF_M5,LOG_M2,LOG_M3,LOG_M4,LOG_M7,LOG_M9
0,0,6,407438,1,3,S1F0,1,0,26953834,0,0,2.079442,0.000000,3.970292,0.0,2.079442
1,0,6,403174,1,3,S1F0,1,0,7671335,0,0,0.000000,1.386294,0.000000,0.0,0.000000
2,0,12,237394,1,3,S1F0,1,0,21661996,0,0,0.000000,0.000000,0.000000,0.0,0.000000
3,0,6,410186,1,3,S1F0,1,0,9961753,0,0,0.000000,0.000000,0.000000,0.0,0.000000
4,0,15,313173,1,3,S1F0,1,0,16996310,0,0,0.000000,0.000000,0.000000,0.0,1.386294


In [7]:
CATEGORICAL_COLUMNS = ["SECTOR"]
CATEGORICAL_COLUMNS_OE = ["SECTOR_OE"]
NUMERICAL_COLUMNS = ['METRIC5','METRIC6','DAY','DAY_WEEK','OP_PERIOD','MNW1','DIF_M6','DIF_M5','LOG_M2',
 'LOG_M3','LOG_M4','LOG_M7','LOG_M9']
LABEL_COLUMNS = ["FAILURE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [8]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [9]:
train_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

Create a pipeline with preprocessing steps and model definition

In [12]:
pipeline = Pipeline(
    steps=[
            (
                "OHE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                SGDClassifier(
                input_cols=NUMERICAL_COLUMNS + CATEGORICAL_COLUMNS_OE + ['DEV_RECONNECTED'],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [13]:
pipeline.fit(train_df)

The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy==1.24.3'. Your UDF might not work when the package version is different between the server and your local environment.


In [ ]:
result = pipeline.predict(test_df)

In [8]:
sf_df1 = session.table("BANK_CUSTOMER_CHURN_PREDICTION").drop('ROW')

In [9]:
CATEGORICAL_COLUMNS = ["COUNTRY", "GENDER"]
CATEGORICAL_COLUMNS_OE = ["COUNTRY_OE", "GENDER_OE"]
NUMERICAL_COLUMNS = ["CREDIT_SCORE", "AGE", "TENURE", "BALANCE", "PRODUCTS_NUMBER", "ESTIMATED_SALARY"]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [10]:
pipeline1 = Pipeline(
    steps=[
            (
                "OE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                XGBClassifier(
                input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS + ["CREDIT_CARD"],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [11]:
train_df1, test_df1 = sf_df1.random_split([0.8,0.2])

In [12]:
pipeline1.fit(train_df1)

The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy==1.24.3'. Your UDF might not work when the package version is different between the server and your local environment.
